In [13]:
import os
import json

f = open('config.json')
config = json.load(f)

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS_SECRET_ACCESS_KEY']
os.environ['AWS_DEFAULT_REGION'] = config['AWS_DEFAULT_REGION']

In [9]:
import yfinance as yf
import pandas as pd
import boto3
import awswrangler as wr
import datetime

In [10]:
tickers = yf.Tickers('MSFT META')

histories = []
for symb in tickers.tickers:
    daily = tickers.tickers[symb].history(period="1d")
    daily['Ticker'] = symb
    daily = daily.reset_index()
    daily['Date'] = daily['Date'].dt.to_pydatetime()[0].strftime("%Y-%m-%d")
    daily = daily[['Ticker', 'Open', 'High', 'Low', 'Close', 'Volume', 'Date']]
    histories.append(daily)
    
data = pd.concat(histories)

In [14]:
todays_date = datetime.datetime.today().strftime("%Y-%m-%d")

# Storing data on Data Lake
wr.s3.to_parquet(
    df=data.copy(),
    path="s3://yfinance-test/daily/",
    dataset=True,
    database="yfinance",
    table="history_daily",
    filename_prefix=todays_date
)

{'paths': ['s3://yfinance-test/daily/2022-10-30ac2fc67a4d324c5c9182073e25663062.snappy.parquet'],
 'partitions_values': {}}